# Visualizing the Magical Core of D&D Classes

### Installing pyvis and importing required libraries

In [238]:
!pip install pyvis

In [239]:
import math
import pandas as pd
import networkx as nx
from pyvis.network import Network

### Loading and preaparing dataset

In [240]:
df = pd.read_csv('dnd-spells.csv')
df.head()

,name,classes,level,school,cast_time,range,duration,verbal,somatic,material,material_cost,description
0,Acid Splash,"Artificer, Sorcerer, Wizard",0,Conjuration,1 Action,60 Feet,Instantaneous,1,1,0,NaN,You hurl a bubble of acid. Choose one creature...
1,Blade Ward,"Bard, Sorcerer, Warlock, Wizard",0,Abjuration,1 Action,Self,1 round,1,1,0,NaN,You extend your hand and trace a sigil of ward...
2,Booming Blade,"Artificer, Sorcerer, Warlock, Wizard",0,Evocation,1 Action,Self (5-foot radius),1 round,0,1,1,a melee weapon worth at least 1 sp,You brandish the weapon used in the spell’s ca...
3,Chill Touch,"Sorcerer, Warlock, Wizard",0,Necromancy,1 Action,120 Feet,1 round,1,1,0,NaN,"You create a ghostly, skeletal hand in the spa..."
4,Control Flames,"Druid, Sorcerer, Wizard",0,Transmutation,1 Action,60 Feet,Instantaneous or 1 hour,0,1,0,NaN,You choose nonmagical flame that you can see w...


In [241]:
# creating a classList column by spliting the classes into list
df['classList'] = df['classes'].str.split(',')

# Focusing only on needed cloumns
cloumn_to_keep = ['name', 'classes', 'classList', 'school']
df = df[cloumn_to_keep]

df.head()

,name,classes,classList,school
0,Acid Splash,"Artificer, Sorcerer, Wizard","[Artificer, Sorcerer, Wizard]",Conjuration
1,Blade Ward,"Bard, Sorcerer, Warlock, Wizard","[Bard, Sorcerer, Warlock, Wizard]",Abjuration
2,Booming Blade,"Artificer, Sorcerer, Warlock, Wizard","[Artificer, Sorcerer, Warlock, Wizard]",Evocation
3,Chill Touch,"Sorcerer, Warlock, Wizard","[Sorcerer, Warlock, Wizard]",Necromancy
4,Control Flames,"Druid, Sorcerer, Wizard","[Druid, Sorcerer, Wizard]",Transmutation


### Create a color platte for Schools of magic

There are 8 schools of magic in D&D
To understand and analyse the data different color would be applied to them

In [242]:
unique_schools = df['school'].unique()

color_palette = [
    '#ff69b4', '#00bfff', '#ff4500', '#dc143c',
    '#9932cc', '#006400', '#ffd700', '#ffffff'
]

color_map = {school: color for school, color in zip(unique_schools, color_palette)}

print("Color map created successfully:")
print(color_map)

Color map created successfully:
{'Conjuration': '#ff69b4', 'Abjuration': '#00bfff', 'Evocation': '#ff4500', 'Necromancy': '#dc143c', 'Transmutation': '#9932cc', 'Enchantment': '#006400', 'Divination': '#ffd700', 'Illusion': '#ffffff'}


### Taking User Input and preparing the graph for visualization

In [243]:

# removing comma and whitespace
df['classes_list'] = df['classes'].str.split(',')
df['classes_list'] = df['classes_list'].apply(lambda class_list: [cls.strip() for cls in class_list])

all_classes = set()
for classes in df['classes_list']:
    all_classes.update(classes)

all_classes = sorted(list(all_classes))

print("List of D&D Classes:")
for i in range(len(all_classes)):
    print(f"{i+1}. {all_classes[i]}")

class_to_look_for = input("Enter the name of class you want to see the spellbook of: ").strip().capitalize()
class_df = df[df['classList'].apply(lambda lst: class_to_look_for in lst)].copy()
class_df.head()

List of D&D Classes:
1. Artificer
2. Bard
3. Cleric
4. Druid
5. Paladin
6. Ranger
7. Sorcerer
8. Warlock
9. Wizard
Enter the name of class you want to see the spellbook of: Artificer


,name,classes,classList,school,classes_list
0,Acid Splash,"Artificer, Sorcerer, Wizard","[Artificer, Sorcerer, Wizard]",Conjuration,"[Artificer, Sorcerer, Wizard]"
2,Booming Blade,"Artificer, Sorcerer, Warlock, Wizard","[Artificer, Sorcerer, Warlock, Wizard]",Evocation,"[Artificer, Sorcerer, Warlock, Wizard]"
6,Dancing Lights,"Artificer, Bard, Sorcerer, Wizard","[Artificer, Bard, Sorcerer, Wizard]",Evocation,"[Artificer, Bard, Sorcerer, Wizard]"
11,Fire Bolt,"Artificer, Sorcerer, Wizard","[Artificer, Sorcerer, Wizard]",Evocation,"[Artificer, Sorcerer, Wizard]"
13,Frostbite,"Artificer, Druid, Sorcerer, Warlock, Wizard","[Artificer, Druid, Sorcerer, Warlock, Wizard]",Evocation,"[Artificer, Druid, Sorcerer, Warlock, Wizard]"


Calculating exclusivity for arranging nodes

In [244]:
class_df['class_count'] = class_df['classList'].apply(len)
class_df.head()

Exclusivity_levels = sorted(class_df['class_count'].unique())

Graph creation

In [245]:
G = nx.Graph()
G.add_node(class_to_look_for, size=40, color='#fcfcfc', shape='star', label=class_to_look_for)

for index, row in class_df.iterrows():
  school = row['school']
  color = color_map.get(school, '#cccccc')

  #Calculate size based on exclusivity, Spells known by fewer classes will be larger.

  node_size = 40 - (row['class_count']**1.5 * 2.5)
  node_size = max(5, node_size)

  G.add_node(row['name'],
  school=school,
  color=color,
  size=node_size,
  title=f"School: {school} | Known by: {row['class_count']} classes")

  G.add_edge(class_to_look_for, row['name'], color='#cccccc', width=0.5)
spell_list = list(class_df.itertuples(index=False))
for i in range(len(spell_list)):
  for j in range(i + 1, len(spell_list)):
    spell1 = spell_list[i]
    spell2 = spell_list[j]
    if spell1.school == spell2.school:
      G.add_edge(spell1.name, spell2.name, color=color_map.get(spell1.school))
print(f" Graph for {class_to_look_for} created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

 Graph for Artificer created with 92 nodes and 1006 edges.


### Visualisation

In [246]:
net_hub = Network(
        notebook=True,
        height="800px",
        width="100%",
        bgcolor="#222222",
        font_color="white",
        cdn_resources='in_line'
    )

net_hub.from_nx(G)
net_hub.force_atlas_2based(gravity=-80, spring_length=150)


file_name = f"{class_to_look_for.lower()}.html"
net_hub.save_graph(file_name)

print(f"\nGenerated Graph for {class_to_look_for}. Please download '{file_name}'")


Generated Graph for Artificer. Please download 'artificer.html'
